In [2]:
import pandas as pd

In [3]:
from tabulate import tabulate

def informacion_dataframe(df):
    # Lista para almacenar información por columna
    info_columnas = []
    
    # Iterar sobre cada columna
    for columna in df.columns:
        # Tipo de dato de la columna
        tipo_dato = df[columna].dtype
        
        # Contar valores nulos
        nulos = df[columna].isnull().sum()
        # Calcular porcentaje de nulos
        porcentaje_nulos = (nulos / len(df)) * 100
        
        # Contar valores duplicados
        duplicados = df.duplicated(subset=columna).sum()
        # Calcular porcentaje de duplicados
        porcentaje_duplicados = (duplicados / len(df)) * 100
        
        # Cantidad de filas de esa columna
        cantidad_filas = len(df[columna])
        
        # Agregar información a la lista
        info_columnas.append([columna, tipo_dato, cantidad_filas, nulos, porcentaje_nulos, duplicados, porcentaje_duplicados])
    
    # Crear DataFrame con la información
    df_info = pd.DataFrame(info_columnas, columns=["Columna", "Tipo de Dato", "Cantidad de Filas", "Nulos", "% Nulos", "Duplicados", "% Duplicados"])
    
    # Imprimir la tabla
    print(tabulate(df_info, headers='keys', tablefmt='fancy_grid', showindex=False))


#### Se crean los dataframes a partir de los datasets

In [7]:
# Especifica la ruta donde se encuentran los archivos
ruta = "Data/Yelp/"


### Vista preliminar de dataset bussiness

In [13]:
# Cargar el dataframe de business
df_business = pd.read_csv(ruta + "business_filtered_by_state.csv")

In [11]:
## FORMA AUXILIAR DE ELIMINAR EL INDEX
# Eliminar la primera columna
df_business_sin_primera_columna = df_business.iloc[:, 1:]

# Guardar el DataFrame sin la primera columna en un nuevo archivo CSV
ruta_nueva = ruta + "business_filtered_by_state.csv"
df_business_sin_primera_columna.to_csv(ruta_nueva, index=False)

In [23]:
df_business.shape

(39247, 14)

In [24]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [14]:
informacion_dataframe(df_business)

╒══════════════╤════════════════╤═════════════════════╤═════════╤════════════╤══════════════╤════════════════╕
│ Columna      │ Tipo de Dato   │   Cantidad de Filas │   Nulos │    % Nulos │   Duplicados │   % Duplicados │
╞══════════════╪════════════════╪═════════════════════╪═════════╪════════════╪══════════════╪════════════════╡
│ business_id  │ object         │               39247 │       0 │  0         │            0 │        0       │
├──────────────┼────────────────┼─────────────────────┼─────────┼────────────┼──────────────┼────────────────┤
│ name         │ object         │               39247 │       0 │  0         │         6940 │       17.6829  │
├──────────────┼────────────────┼─────────────────────┼─────────┼────────────┼──────────────┼────────────────┤
│ address      │ object         │               39247 │    1341 │  3.41682   │         3550 │        9.04528 │
├──────────────┼────────────────┼─────────────────────┼─────────┼────────────┼──────────────┼────────────────┤
│

###
* Los elementos duplicados, en principio no presentan un problema en la mayoría de las columnas, ya que tiene sentido que haya varios negocios en la misma ciudad o estado al igual que es posible que se repitan categorias, esrellas, horarios, atributos, conteo de reseñas o codigo postal para un mismo negocio. Sin embargo, podemos ver que ocurre en filas con nombres o ubicaciones repetidas.

#### Revisamos que las duplicaciones en nombres sean efectivamente locales con el mismo nombre ubicados en distintos lugares

In [21]:
# Identificar filas duplicadas basadas en las columnas "name" y "city"
filas_duplicadas = df_business[df_business.duplicated(subset=["name", "latitude", "longitude"], keep=False)]

# Ordenar el resultado por el nombre y la ciudad para facilitar la comparación
filas_duplicadas_ordenadas = filas_duplicadas.sort_values(by=["name", "city"])

filas_duplicadas_ordenadas

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
18404,nsh6p0WVqGB8OhnM6_CKLg,Century 20 El Con and XD,3601 E Broadway,Tucson,FL,85716,32.224841,-110.916351,3.0,19,1,"{'GoodForKids': 'True', 'HappyHour': 'False', ...","Arts & Entertainment, Cinema",NaN
28240,3uLgwr0qeCNMjKenHJwPGQ,Century 20 El Con and XD,3601 E Broadway Blvd,Tucson,FL,85716,32.224841,-110.916351,3.5,72,1,{'GoodForKids': 'True'},"Cinema, Event Planning & Services, Arts & Ente...",NaN
27153,BcErXCI_VKfkSfpdgzsOYw,Chimney Safe,NaN,Reno,FL,89509,39.495379,-119.823655,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Inspectors, Chimney Sweeps, Home Services","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
28494,ZwkO9bkvXwbtTJ1DfGFzKg,Chimney Safe,NaN,Reno,FL,89509,39.495379,-119.823655,3.0,6,0,NaN,"Chimney Sweeps, Home Services","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
8606,Mk2184ZyG1PcS1rMFxvrUA,Tap Room On 19th,2400 S 19th St,Philadelphia,NV,19145,39.921596,-75.178679,4.0,16,0,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...","Pubs, Comfort Food, Restaurants, Gastropubs, N...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ..."
14530,kL98j83h3QFUoSBBevCHOw,Tap Room On 19th,2400 S 19th St,Philadelphia,FL,19145,39.921596,-75.178679,4.0,183,1,"{'HasTV': 'True', 'RestaurantsGoodForGroups': ...","Restaurants, Gastropubs, American (New)","{'Monday': '0:0-0:0', 'Tuesday': '15:0-22:0', ..."


Notamos que hay unas pocas filas que parecen referir al mismo restaurante ya que el nombre y su ubicacion (lat y long) es exactamente la misma, a pesar de estar registrados con un id de negocio distinto. Ya que son pocos casos, nos quedaremos con las filas que tengan mayor conteo de reviews asociadas para resolver estos duplicados de manera simple preservando la mayor cantidad de reviews posibles.

In [27]:
# Ordenar las filas duplicadas por el número de revisiones en orden descendente
filas_duplicadas_ordenadas = filas_duplicadas.sort_values(by="review_count", ascending=False)

# Identificar las filas a eliminar (todas excepto la primera ocurrencia)
filas_a_eliminar = filas_duplicadas_ordenadas.drop_duplicates(subset=["name", "latitude", "longitude"], keep="first")

# Eliminar las filas duplicadas del DataFrame original
df_business_sin_duplicados = df_business.drop(filas_a_eliminar.index)

df_business = df_business_sin_duplicados

In [28]:
# Identificar filas duplicadas basadas en las columnas "name" y "city"
filas_duplicadas = df_business[df_business.duplicated(subset=["name", "latitude", "longitude"], keep=False)]

# Ordenar el resultado por el nombre y la ciudad para facilitar la comparación
filas_duplicadas_ordenadas = filas_duplicadas.sort_values(by=["name", "city"])

filas_duplicadas_ordenadas

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours


Ahora revisamos que ocurre en filas con latitud y longitud iguales simultaneamente

In [33]:
# Filtrar filas duplicadas basadas en las columnas "latitude" y "longitude"
filas_duplicadas_lat_long = df_business[df_business.duplicated(subset=["latitude", "longitude"], keep=False)]
# Ordenar el resultado por el nombre y la ciudad para facilitar la comparación
filas_duplicadas_ordenadas = filas_duplicadas_lat_long.sort_values(by=["latitude", "longitude"])
filas_duplicadas_ordenadas

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
12037,aYrmK2xwh2iIZB4taWNwtg,Sea Worthy Fish + Bar,1110 Pinellas Bayway S,Tierra Verde,FL,33715,27.675726,-82.726385,4.5,58,1,"{'Ambience': ""{'touristy': None, 'hipster': No...","Bars, Breakfast & Brunch, Restaurants, Cocktai...","{'Tuesday': '16:30-21:0', 'Wednesday': '16:30-..."
18338,cEAjDyP3Zwdz-c3jU0Oq3w,Tierra Verde Property Management,1110 Pinellas Bayway S,Saint Petersburg,FL,33715,27.675726,-82.726385,1.5,6,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Property Management, Real Estate, Home Services",NaN
1825,9qccR7IOPYKgjEOOqT8PdQ,Lil Swimmers Aquatics,NaN,Ruskin,NV,33570,27.695851,-82.453611,4.5,7,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Education, Local Services, Specialty Schools, ...",NaN
33281,hDtwZPO-GFAjrJw2kizgtQ,A Breathe of Fresh Aire,NaN,Ruskin,FL,33570,27.695851,-82.453611,3.5,9,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Heating & Air Conditioning/HVAC, Local Service...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
19747,DiJ1u1svwBK8SE59UTAhug,Nava Yoga Studio,"5901 Sun Blvd W, Ste 120",St. Petersburg,FL,33715,27.713162,-82.714213,5.0,5,1,"{'ByAppointmentOnly': 'False', 'BikeParking': ...","Yoga, Meditation Centers, Active Life, Medical...","{'Monday': '0:0-0:0', 'Tuesday': '6:30-20:30',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31543,tVJCBzthLTqe7FlFv6WLAA,Golden Dumpling Restaurant,10939-101 Street,Edmonton,CA,T5H 2S7,53.555712,-113.493526,3.5,7,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForM...","Restaurants, Chinese, Noodles, Dumplings","{'Monday': '11:0-21:0', 'Wednesday': '11:0-21:..."
11265,Wc2dOe5VdFoYwc-0wvkhwg,Italian Centre Shop,10878 95 Street NW,Edmonton,FL,T5H 2E4,53.556265,-113.485181,4.5,114,1,"{'RestaurantsReservations': 'False', 'HasTV': ...","Italian, Grocery, Delis, Restaurants, Food, In...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-21:0', '..."
17874,dq-02omJz_ufg1F-BME0kg,Spinelli's Bar Italia,10878 95 St,Edmonton,FL,T5H 2E4,53.556265,-113.485181,4.0,25,1,"{'RestaurantsTakeOut': 'True', 'Alcohol': ""'be...","Coffee & Tea, Desserts, Restaurants, Sandwiche...","{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ..."
10641,25uEEaWkWO5PHg_bh9BZwg,Thai Express,"109 Street & Princess Elizabeth Avenue, Suite ...",Edmonton,CA,T5G 3A6,53.564194,-113.508499,2.5,8,1,"{'RestaurantsTakeOut': 'True', 'RestaurantsDel...","Restaurants, Thai",NaN


Observamos que es posible que se comparta la latitud y longitud y referir a negocios distintos, ubicados dentrode un conglomerado. Por lo que en principio no hacemos nada con estas duplicaciones de latitud y longitud simultaneas.

Ahora tomaremos una decision respecto a los elementos nulos.

### Vista preliminar de dataset de checkin

In [ ]:
# Cargar el dataframe de checkin
df_checkin =  pd.read_csv(ruta + "checkin.csv")

In [ ]:
informacion_dataframe(df_checkin)

### Vista preliminar de dataset review

In [35]:
# Cargar el dataframe de review
df_review =  pd.read_csv("Data/Yelp/reviews_filtered_by_state.csv")

In [36]:
informacion_dataframe(df_review)

╒═════════════╤════════════════╤═════════════════════╤═════════╤═══════════╤══════════════╤════════════════╕
│ Columna     │ Tipo de Dato   │   Cantidad de Filas │   Nulos │   % Nulos │   Duplicados │   % Duplicados │
╞═════════════╪════════════════╪═════════════════════╪═════════╪═══════════╪══════════════╪════════════════╡
│ review_id   │ object         │             1820145 │       0 │         0 │            0 │       0        │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ user_id     │ object         │             1820145 │       0 │         0 │       975179 │      53.577    │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ business_id │ object         │             1820145 │       0 │         0 │      1780898 │      97.8437   │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ stars       │ int

* No se observan nulos en los datos.
* En las columnas "user_id", "bussiness_id", "stars", "useful", "funny", "cool", "text" y "date" se observan elementos duplicados. Esto no se trata de un error, ya que los valores de estas columnas pueden repetirse en cada reseña. Por ejemplo, puede haber un mismo usuario que hizo reseñas para distintos negocios. O un mismo negocio fue reseñado por distintos usuarios. Las columnas asociadas a las opiniones, texto y fecha tampoco presentan ningun problema respecto a su repeticion.



Optamos por eliminar las columnas "funny" y "cool" ya que solo representan una caracteristica que se le da a una reseña y no presentan valor para los objetivos del proyecto.

In [37]:
# Utiliza iloc para seleccionar todas las columnas excepto "funny" y "cool"
df_review = df_review.iloc[:, ~df_review.columns.isin(['funny', 'cool'])]

In [38]:
informacion_dataframe(df_review)

╒═════════════╤════════════════╤═════════════════════╤═════════╤═══════════╤══════════════╤════════════════╕
│ Columna     │ Tipo de Dato   │   Cantidad de Filas │   Nulos │   % Nulos │   Duplicados │   % Duplicados │
╞═════════════╪════════════════╪═════════════════════╪═════════╪═══════════╪══════════════╪════════════════╡
│ review_id   │ object         │             1820145 │       0 │         0 │            0 │       0        │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ user_id     │ object         │             1820145 │       0 │         0 │       975179 │      53.577    │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ business_id │ object         │             1820145 │       0 │         0 │      1780898 │      97.8437   │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ stars       │ int

Viendo el "review_id", a pesar de no haber duplicados de forma aparente, notamos que puede haber reviews iguales hechas por el mismo usuario, pero que se diferencien solo en alguna/s categoría/s, manteniendo el mismo "review_id". Procedemos a ver esta situación.

In [39]:
duplicated_rows = df_review[df_review.duplicated(subset=['user_id', 'text', 'date', 'business_id', 'stars'], keep=False)]
duplicated_rows

,review_id,user_id,business_id,stars,useful,text,date
325423,gZU6syBqLk210QhnwWbSyw,6d4TXsdOZAsw8_h1r730uA,t1H2xPVsbTRu48bZ5TgItw,1,0,I was home for the holidays and staying with m...,2010-12-27 23:22:50
332032,6FJ-pUVQsMFHhW4RJltm6w,6d4TXsdOZAsw8_h1r730uA,t1H2xPVsbTRu48bZ5TgItw,1,3,I was home for the holidays and staying with m...,2010-12-27 23:22:50
362085,uN7sRdsxIJgmD5REegSpBA,6d4TXsdOZAsw8_h1r730uA,t1H2xPVsbTRu48bZ5TgItw,1,1,I was home for the holidays and staying with m...,2010-12-27 23:22:50
693106,lzBHkMZayoGSpIPDOIIEug,3B5wkOgsrjBjXRpGdrmTSA,n3Q7DXPyBzGjPK1l09SomQ,4,0,"Fox Rehab at-home in Virgina-Northern, Fairfax...",2021-02-01 17:04:06
722113,QacdXke_oZAD662oeFyUxg,3B5wkOgsrjBjXRpGdrmTSA,n3Q7DXPyBzGjPK1l09SomQ,4,0,"Fox Rehab at-home in Virgina-Northern, Fairfax...",2021-02-01 17:04:06
1425452,XjlaQ5ImGg1uL2XelQA2xw,xZ0ioQJ0Jyo_Uc0FzmyFQQ,O_kAd89gPls-dXda40NWzA,5,0,My son and I moved here from South Carolina a ...,2019-11-22 07:18:46
1438330,8JcvaBpFPAe7lIVSazi0hQ,xZ0ioQJ0Jyo_Uc0FzmyFQQ,O_kAd89gPls-dXda40NWzA,5,0,My son and I moved here from South Carolina a ...,2019-11-22 07:18:46


De esta forma podemos ver que hay reseñas repetidas, probablemente por errores del servidor en capturar las reviews o incluso por algun comportamiento del usuario.

Procedemos a eliminar las reseñas repetidas

In [40]:
# Eliminar las filas duplicadas conservando solo la primera ocurrencia
df_review = df_review.drop_duplicates(subset=['user_id', 'text', 'date', 'business_id', 'stars'], keep='first')

In [41]:
informacion_dataframe(df_review)

╒═════════════╤════════════════╤═════════════════════╤═════════╤═══════════╤══════════════╤════════════════╕
│ Columna     │ Tipo de Dato   │   Cantidad de Filas │   Nulos │   % Nulos │   Duplicados │   % Duplicados │
╞═════════════╪════════════════╪═════════════════════╪═════════╪═══════════╪══════════════╪════════════════╡
│ review_id   │ object         │             1820141 │       0 │         0 │            0 │       0        │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ user_id     │ object         │             1820141 │       0 │         0 │       975175 │      53.5769   │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ business_id │ object         │             1820141 │       0 │         0 │      1780894 │      97.8437   │
├─────────────┼────────────────┼─────────────────────┼─────────┼───────────┼──────────────┼────────────────┤
│ stars       │ int

Se exporta el dataframe limpio como csv

In [42]:
df_review.to_csv(ruta+'df_review_clean.csv', index=False)

### Vista preliminar de dataset tip

In [ ]:
# Cargar el dataframe tip
df_tip =  pd.read_csv(ruta + "tip.csv")

In [ ]:
informacion_dataframe(df_tip)

### Vista preliminar de dataset user

In [ ]:
# Cargar el dataframe de user
df_user =  pd.read_csv(ruta + "user.csv")

In [ ]:
informacion_dataframe(df_user)